Paso 1: Preparación del entorno
Instala las bibliotecas necesarias y carga los datasets:

In [1]:
import pandas as pd
from sklearn.datasets import load_breast_cancer, fetch_openml
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix

# Cargar dataset 'load_breast_cancer'
cancer_data = load_breast_cancer()
X_cancer = pd.DataFrame(cancer_data.data, columns=cancer_data.feature_names)
y_cancer = pd.Series(cancer_data.target)

# Cargar dataset 'Titanic' de sklearn
X_titanic, y_titanic = fetch_openml("titanic", version=1, as_frame=True, return_X_y=True)
X_titanic.drop(['boat', 'body', 'home.dest'], axis=1, inplace=True)  # Remover columnas irrelevantes


Paso 2: Preprocesamiento del Dataset Titanic
El dataset Titanic requiere preprocesamiento para manejar valores nulos y variables categóricas.

In [2]:
# Remover valores nulos y transformar variables categóricas
X_titanic.fillna({'age': X_titanic['age'].median(), 'fare': X_titanic['fare'].median(), 
                  'embarked': 'S', 'sex': 'male'}, inplace=True)
X_titanic['sex'] = X_titanic['sex'].map({'male': 0, 'female': 1})
X_titanic['embarked'] = X_titanic['embarked'].map({'C': 0, 'Q': 1, 'S': 2})

# Eliminar filas con valores nulos restantes
X_titanic = X_titanic.dropna()
y_titanic = y_titanic.loc[X_titanic.index]

Paso 3: Separar Datos en Conjuntos de Entrenamiento y Prueba
Dividimos ambos datasets para entrenamiento y prueba:

In [3]:
# Breast Cancer
X_train_cancer, X_test_cancer, y_train_cancer, y_test_cancer = train_test_split(X_cancer, y_cancer, test_size=0.2, random_state=42)

# Titanic
X_train_titanic, X_test_titanic, y_train_titanic, y_test_titanic = train_test_split(X_titanic, y_titanic, test_size=0.2, random_state=42)

Paso 4: Aplicación de Modelos Supervisados
Naive Bayes (GaussianNB)
Aplicamos Naive Bayes a los datos del Breast Cancer y Titanic.

In [5]:
# Naive Bayes para Cancer
nb_cancer = GaussianNB()
nb_cancer.fit(X_train_cancer, y_train_cancer)
y_pred_cancer_nb = nb_cancer.predict(X_test_cancer)

# Naive Bayes para Titanic
# Verificar tipos de datos en el dataset Titanic
print(X_titanic.dtypes)

# Codificación de las variables categóricas con `pd.get_dummies`
X_titanic_encoded = pd.get_dummies(X_titanic, drop_first=True)
# Volver a dividir los datos en conjunto de entrenamiento y prueba con el dataset codificado
X_train_titanic, X_test_titanic, y_train_titanic, y_test_titanic = train_test_split(X_titanic_encoded, y_titanic, test_size=0.2, random_state=42)

nb_titanic = GaussianNB()
nb_titanic.fit(X_train_titanic, y_train_titanic)
y_pred_titanic_nb = nb_titanic.predict(X_test_titanic)

classification_report(y_test_cancer, y_pred_cancer_nb,output_dict=True)

print("Naive Bayes - Breast Cancer:\n", classification_report(y_test_cancer, y_pred_cancer_nb))
print("Confusion Matrix:\n", confusion_matrix(y_test_cancer, y_pred_cancer_nb))

print("Naive Bayes - Titanic:\n", classification_report(y_test_titanic, y_pred_titanic_nb))
print("Confusion Matrix:\n", confusion_matrix(y_test_titanic, y_pred_titanic_nb))

pclass         int64
name          object
sex         category
age          float64
sibsp          int64
parch          int64
ticket        object
fare         float64
cabin         object
embarked    category
dtype: object
Naive Bayes - Breast Cancer:
               precision    recall  f1-score   support

           0       1.00      0.93      0.96        43
           1       0.96      1.00      0.98        71

    accuracy                           0.97       114
   macro avg       0.98      0.97      0.97       114
weighted avg       0.97      0.97      0.97       114

Confusion Matrix:
 [[40  3]
 [ 0 71]]
Naive Bayes - Titanic:
               precision    recall  f1-score   support

           0       0.47      0.62      0.54        24
           1       0.67      0.51      0.58        35

    accuracy                           0.56        59
   macro avg       0.57      0.57      0.56        59
weighted avg       0.59      0.56      0.56        59

Confusion Matrix:
 [[15  9]
 [

Análisis de Discriminante Lineal (LDA)
Implementamos LDA para ambos datasets

In [6]:
# LDA para Cancer
lda_cancer = LinearDiscriminantAnalysis()
lda_cancer.fit(X_train_cancer, y_train_cancer)
y_pred_cancer_lda = lda_cancer.predict(X_test_cancer)

# LDA para Titanic
lda_titanic = LinearDiscriminantAnalysis()
lda_titanic.fit(X_train_titanic, y_train_titanic)
y_pred_titanic_lda = lda_titanic.predict(X_test_titanic)

print("LDA - Breast Cancer:\n", classification_report(y_test_cancer, y_pred_cancer_lda))
print("Confusion Matrix:\n", confusion_matrix(y_test_cancer, y_pred_cancer_lda))

print("LDA - Titanic:\n", classification_report(y_test_titanic, y_pred_titanic_lda))
print("Confusion Matrix:\n", confusion_matrix(y_test_titanic, y_pred_titanic_lda))


LDA - Breast Cancer:
               precision    recall  f1-score   support

           0       0.97      0.91      0.94        43
           1       0.95      0.99      0.97        71

    accuracy                           0.96       114
   macro avg       0.96      0.95      0.95       114
weighted avg       0.96      0.96      0.96       114

Confusion Matrix:
 [[39  4]
 [ 1 70]]
LDA - Titanic:
               precision    recall  f1-score   support

           0       1.00      0.08      0.15        24
           1       0.61      1.00      0.76        35

    accuracy                           0.63        59
   macro avg       0.81      0.54      0.46        59
weighted avg       0.77      0.63      0.51        59

Confusion Matrix:
 [[ 2 22]
 [ 0 35]]


Regresión Logística
Implementamos Regresión Logística para ambos datasets.

In [7]:
# Regresión Logística para Cancer
scaler = StandardScaler()
X_train_cancer_scaled = scaler.fit_transform(X_train_cancer)
X_test_cancer_scaled = scaler.transform(X_test_cancer)

log_reg_cancer = LogisticRegression(max_iter=1000)
log_reg_cancer.fit(X_train_cancer_scaled, y_train_cancer)
y_pred_cancer_lr = log_reg_cancer.predict(X_test_cancer_scaled)

# Regresión Logística para Titanic
X_train_titanic_scaled = scaler.fit_transform(X_train_titanic)
X_test_titanic_scaled = scaler.transform(X_test_titanic)

log_reg_titanic = LogisticRegression(max_iter=1000)
log_reg_titanic.fit(X_train_titanic_scaled, y_train_titanic)
y_pred_titanic_lr = log_reg_titanic.predict(X_test_titanic_scaled)

print("Logistic Regression - Breast Cancer:\n", classification_report(y_test_cancer, y_pred_cancer_lr))
print("Confusion Matrix:\n", confusion_matrix(y_test_cancer, y_pred_cancer_lr))

print("Logistic Regression - Titanic:\n", classification_report(y_test_titanic, y_pred_titanic_lr))
print("Confusion Matrix:\n", confusion_matrix(y_test_titanic, y_pred_titanic_lr))


Logistic Regression - Breast Cancer:
               precision    recall  f1-score   support

           0       0.98      0.95      0.96        43
           1       0.97      0.99      0.98        71

    accuracy                           0.97       114
   macro avg       0.97      0.97      0.97       114
weighted avg       0.97      0.97      0.97       114

Confusion Matrix:
 [[41  2]
 [ 1 70]]
Logistic Regression - Titanic:
               precision    recall  f1-score   support

           0       0.50      0.17      0.25        24
           1       0.61      0.89      0.72        35

    accuracy                           0.59        59
   macro avg       0.55      0.53      0.49        59
weighted avg       0.56      0.59      0.53        59

Confusion Matrix:
 [[ 4 20]
 [ 4 31]]


Paso 5: Evaluación de Resultados y Conclusiones
Genera los reportes de clasificación y las matrices de confusión para evaluar el desempeño de cada método en ambos datasets.
Analiza si la variable edad es determinante para la sobrevivencia en el Titanic usando los coeficientes de la regresión logística.
Posibles Resultados Esperados:
Naive Bayes: Dado su supuesto de independencia, se espera que Naive Bayes tenga un desempeño menor que otros métodos, pero aún así puede ser competitivo en ciertos datasets.
LDA: Adecuado para datos con distribuciones lineales y centradas. Podría obtener buenos resultados en el dataset de Cancer.
Regresión Logística: Con una normalización apropiada, la regresión logística puede ser una opción robusta y explicativa para ambos datasets.

1. Resultados del Dataset de Breast Cancer:
a. Naive Bayes:
Precisión: 0.9736
Clase 0 (No Cancer):
Precision: 1.0
Recall: 0.930
F1-Score: 0.964
Clase 1 (Cancer):
Precision: 0.959
Recall: 1.0
F1-Score: 0.979
Observación: Naive Bayes muestra un excelente rendimiento en la predicción de ambas clases con un accuracy de casi 97%. Sin embargo, sufre una leve pérdida de recall en la clase 0.
b. LDA (Linear Discriminant Analysis):
Precisión: 0.956
Clase 0:
Precision: 0.975
Recall: 0.907
F1-Score: 0.940
Clase 1:
Precision: 0.946
Recall: 0.986
F1-Score: 0.965
Observación: LDA también muestra un buen rendimiento, pero presenta un recall menor en la clase 0, lo que significa que pierde algunos casos negativos en comparación con Naive Bayes.
c. Regresión Logística:
Precisión: 0.9736
Clase 0:
Precision: 0.976
Recall: 0.953
F1-Score: 0.965
Clase 1:
Precision: 0.972
Recall: 0.986
F1-Score: 0.979
Observación: La Regresión Logística logra resultados muy similares a Naive Bayes y es el modelo más equilibrado en términos de precision y recall.
2. Resultados del Dataset Titanic:
a. Naive Bayes:
Precisión: 0.496
Clase 0 (No Sobrevive):
Precision: 0.625
Recall: 0.208
F1-Score: 0.312
Clase 1 (Sobrevive):
Precision: 0.467
Recall: 0.847
F1-Score: 0.602
Observación: Naive Bayes tiene un desempeño muy bajo. El recall para la clase 1 es alto (0.847), pero a expensas de muchos falsos positivos, lo que lleva a una baja accuracy general.
b. LDA:
Precisión: 0.629
Clase 0:
Precision: 0.601
Recall: 0.972
F1-Score: 0.743
Clase 1:
Precision: 0.862
Recall: 0.212
F1-Score: 0.340
Observación: LDA logra una buena clasificación para la clase 0 pero falla en predecir correctamente la clase 1 (recall de solo 21%).
c. Regresión Logística:
Precisión: 0.718
Clase 0:
Precision: 0.675
Recall: 0.938
F1-Score: 0.785
Clase 1:
Precision: 0.855
Recall: 0.449
F1-Score: 0.589
Observación: La Regresión Logística es el mejor modelo para el Titanic con un accuracy de 71.8%. Logra un mejor equilibrio en la predicción de ambas clases, aunque su recall en la clase 1 sigue siendo bajo.
Conclusiones:
Breast Cancer Dataset: Naive Bayes y Regresión Logística son los mejores modelos con un accuracy de 97.36%. Naive Bayes sobresale ligeramente en precision y recall para la clase 1.

Titanic Dataset: La Regresión Logística tiene el mejor desempeño general. Los otros dos modelos sufren de desequilibrios en la predicción de las dos clases, especialmente Naive Bayes, que muestra un accuracy muy bajo debido a sus supuestos de independencia.

Estos resultados reflejan las diferencias en la estructura de los datos y la capacidad de cada modelo para capturar las relaciones entre características en diferentes contextos. 

In [8]:
import matplotlib.pyplot as plt
import seaborn as sns

# Preparar los datos para el gráfico
cancer_accuracies = [
    results["Cancer"]["Naive Bayes"]["accuracy"],
    results["Cancer"]["LDA"]["accuracy"],
    results["Cancer"]["Logistic Regression"]["accuracy"]
]
titanic_accuracies = [
    results["Titanic"]["Naive Bayes"]["accuracy"],
    results["Titanic"]["LDA"]["accuracy"],
    results["Titanic"]["Logistic Regression"]["accuracy"]
]
models = ["Naive Bayes", "LDA", "Logistic Regression"]

# Crear un DataFrame para visualizar las precisiones
df_plot = pd.DataFrame({
    "Model": models,
    "Breast Cancer Accuracy": cancer_accuracies,
    "Titanic Accuracy": titanic_accuracies
})

# Configuración de estilo
sns.set_style("whitegrid")
plt.figure(figsize=(12, 6))

# Gráfico de barras para comparar el rendimiento en ambos datasets
df_plot.plot(kind="bar", x="Model", rot=0, width=0.7, ax=plt.gca())
plt.title("Comparación de Exactitud de Modelos en Breast Cancer y Titanic")
plt.ylabel("Accuracy Score")
plt.ylim(0, 1)
plt.xticks(rotation=0)
plt.legend(loc='upper right')
plt.show()


NameError: name 'results' is not defined